In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import matplotlib.pyplot as plt
from pathlib import Path
import kagglehub
from functools import lru_cache
from typing import Sequence
from math import log

np.set_printoptions(suppress=True)

In [ ]:
# import
path = Path(kagglehub.dataset_download("mczielinski/bitcoin-historical-data")) / "btcusd_1-min_data.csv"
raw_records = pd.read_csv(
    path,
    usecols=["Open", "High", "Low", "Close", "Volume"],
    #nrows=100_000
)
first_datetime = datetime.fromisoformat(pd.read_csv(path, nrows=1).loc[0, "datetime"])
print("Done")

In [ ]:
# assigning a new index to group later
datetime_index = pd.date_range(
    start=first_datetime,
    freq=timedelta(minutes=1),
    periods=raw_records.index[-1] + 1
)
records = raw_records
records.index = datetime_index
records.index.name = "datetime"
records.rename(columns=str.lower, inplace=True)
records.head()

In [ ]:
# groping
records = records.groupby(records.index.date).agg({
    "open": lambda x: x.iat[0],
    "high": "max",
    "low": "min",
    "close": lambda x: x.iat[-1],
    "volume": "sum"
})
records.index = pd.DatetimeIndex(records.index)
records.tail()

---
**If you held bitcoin for up to 30 days where would your profit lie?**

In [ ]:
# re-running the cell will clear all the cache!
@lru_cache
def profit_quantiles(MAX_SPAN_DAYS: int = 30,
                     QUANTILES: Sequence[float | int] = (0.01, 0.25, 0.5, 0.75, 0.99)) -> pd.DataFrame:
    return_of_hold = pd.DataFrame({k: 0.0 for k in QUANTILES}, index=range(1, MAX_SPAN_DAYS + 1))

    grouped = records["close"].reset_index(drop=True)

    for span in range(2, MAX_SPAN_DAYS + 1):
        res = (grouped
               .rolling(window=span)
               .apply(lambda srs: (srs.iat[-1] / srs.iat[0] - 1) * 100)
               .quantile(QUANTILES)
               )
        return_of_hold.loc[span] = res
    return return_of_hold

In [ ]:
# calculate the profit
MAX_SPAN_DAYS = 30
QUANTILES = (0.01, 0.25, 0.5, 0.75, 0.99)

return_of_hold = profit_quantiles(MAX_SPAN_DAYS=MAX_SPAN_DAYS, QUANTILES=QUANTILES)

In [ ]:
# charting
ax = plt.subplot()
for quantile, data in return_of_hold.items():
    if data.iat[-1] < 0:
        color = "red"
    elif data.iat[-1] == 0:
        color = "grey"
    else:
        color = "green"

    ax.plot(data, color=color)
    ax.text(x=MAX_SPAN_DAYS * 1.02, y=data.iat[-1], s=f"Q {round(quantile, 4)} = {round(data.iat[-1], 2)}%",
            color=color)

ax.axhline(0, linestyle="dashed", color="grey")

ax.set_title("Quantiles of return against days held")
ax.grid()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xlabel("Days held")
ax.set_xlim(1, MAX_SPAN_DAYS * 1.01)

ax.set_yticks(
    np.arange(return_of_hold.min(axis=None) * 0.9, return_of_hold.max(axis=None) * 1.1,
              return_of_hold.mean(axis=None) * 3).round(2)
)
ax.set_ylabel("Return in %");

---
**How does price change correlate with change of trading volume?**

In [ ]:
QUANTILES = [0.05, 0.95]

volume_change = records["volume"].pct_change().dropna()
price_change = records["close"] / records["open"] - 1

price_volume_change = pd.concat([volume_change, price_change], axis="columns")
price_volume_change.rename(columns={"volume": "volume_change", 0: "price_change"}, inplace=True)

mask = price_volume_change.isin([np.nan, np.inf, -np.inf])
price_volume_change[mask] = 0

volume_mask = pd.qcut(price_volume_change["volume_change"], q=QUANTILES).isna().to_numpy()
price_mask = pd.qcut(price_volume_change["price_change"], q=QUANTILES).isna().to_numpy()

general_mask = np.stack([volume_mask, price_mask], axis=1).any(axis=1)

indices_to_drop = price_volume_change.loc[general_mask].index
price_volume_change.drop(indices_to_drop, inplace=True)
price_volume_change.tail()

In [ ]:
# charting
fig, ax = plt.subplots(figsize=(13, 5))
ax.scatter(
    x=price_volume_change["price_change"],
    y=price_volume_change["volume_change"],
    s=0.7,
    color="green"
)

ax.axvline(x=0, linestyle="dashed", color="grey")
ax.axhline(y=0, linestyle="dashed", color="grey")

ax.set_ylabel("Trading colume change")
ax.set_xlabel("Price change")
ax.set_title("Daily price change against change of trading volume (%)");

---
**How does halving affect price and trading volume?**

In [ ]:
halving_dates = (
    date(year=2012, month=11, day=28),
    date(year=2016, month=7, day=9),
    date(year=2020, month=7, day=10),
    date(year=2024, month=4, day=20),
)

grouped = (records
           .groupby([records.index.year, records.index.month])
           .agg({"close": "median", "volume": "sum"})
           .reset_index(drop=True)
           )
grouped.index = pd.date_range(start=first_datetime, freq=timedelta(days=30), periods=len(grouped))

In [ ]:
ax = plt.subplot()
ax.plot(grouped["close"].map(log), color="blue")
ax.set_title("Bitcoin price development after each halving")
ax.set_xlabel("Year")
ax.set_ylabel("Log price in $")

for date in halving_dates:
    ax.axvline(x=date, linestyle="dashed", color="green",label="halving date")

ax.legend(ax.get_children()[:2],["Log price","Halving"]);

---
**How did trading volume per dollar change over the years?**

In [ ]:
grouped = (records
           .groupby([records.index.year, records.index.month])
           .agg({"close": "median", "volume": "sum"})
           .reset_index(drop=True)
           )
grouped.index = pd.date_range(start=first_datetime, freq=timedelta(days=30), periods=len(grouped))
volume_per_dollar = grouped["volume"] / grouped["close"]

In [ ]:
ax = plt.subplot()

ax.plot(volume_per_dollar.map(lambda x: log(x)), color="green")
ax.set_ylabel("log( Trading volume per dollar )")
ax.set_xlabel("Year")
ax.set_title("Trading volume per dollar price over time");